<a href="https://colab.research.google.com/github/JSJeong-me/KOSA-Big-Data_Vision/blob/main/Model/03-NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
from keras.models import Sequential
from keras.layers import Dense

In [3]:
df = pd.read_csv("./credit_cards_dataset.csv")

In [4]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')

In [5]:
df.tail()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
29995,29996,220000.0,1,3,1,39,0,0,0,0,0,0,188948.0,192815.0,208365.0,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,29997,150000.0,1,3,2,43,-1,-1,-1,-1,0,0,1683.0,1828.0,3502.0,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,29998,30000.0,1,2,2,37,4,3,2,-1,0,0,3565.0,3356.0,2758.0,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,29999,80000.0,1,3,1,41,1,-1,0,0,0,-1,-1645.0,78379.0,76304.0,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1
29999,30000,50000.0,1,2,1,46,0,0,0,0,0,0,47929.0,48905.0,49764.0,36535.0,32428.0,15313.0,2078.0,1800.0,1430.0,1000.0,1000.0,1000.0,1


In [6]:
# X = df.drop('Y_Value',axis =1).values
# y = df['Y_Value'].values
X = df.drop('default.payment.next.month',axis =1).values

In [7]:
X.shape

(30000, 24)

In [8]:
y = df['default.payment.next.month'].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split (X,y,test_size=0.3, random_state=42)

In [10]:
y_test.T

array([0, 0, 0, ..., 0, 0, 0])

In [11]:
X_test.shape

(9000, 24)

In [12]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

In [13]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
y_train_categorical = tf.keras.utils.to_categorical(y_train)
y_test_categorical = tf.keras.utils.to_categorical(y_test)

In [15]:
from keras.models import Sequential

#instantiate
model = Sequential()

In [16]:
from keras.layers import Dense

number_inputs = 24
number_units = 60

model.add(Dense(units = number_units, activation ='relu', input_dim=number_inputs))
model.add(Dense(units = 70, activation ='relu')) #second hidden layer
model.add(Dense(units = 60, activation ='relu')) #second hidden layer
#model.add(Dense(units = 50, activation ='relu')) #second hidden layer
#model.add(Dense(units = 40, activation ='relu')) #second hidden layer
#model.add(Dense(units = 30, activation ='relu')) #second hidden layer
#model.add(Dense(units = 20, activation ='relu')) #second hidden layer
model.add(Dense(units = 10, activation ='relu')) #second hidden layer

In [17]:
number_classes =2 ## yes or no
model.add(Dense(units = number_classes, activation = 'sigmoid'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                1500      
_________________________________________________________________
dense_1 (Dense)              (None, 70)                4270      
_________________________________________________________________
dense_2 (Dense)              (None, 60)                4260      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                610       
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 22        
Total params: 10,662
Trainable params: 10,662
Non-trainable params: 0
_________________________________________________________________


In [19]:
#compile the model
model.compile(optimizer = 'adam' ,
             loss = 'binary_crossentropy',
             metrics =['accuracy'])

In [ ]:
#train the model

model.fit(X_train_scaled, y_train_categorical, epochs=50,shuffle = True,verbose =2)

In [21]:
#model.save("neuralnetwork.h5")

In [22]:
#quantify the model
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test_categorical,verbose =2)
print( model_loss )
print (model_accuracy)

282/282 - 0s - loss: 0.6776 - accuracy: 0.7871
0.6776339411735535
0.7871111035346985


F1, Precision Recall, and Confusion Matrix

In [23]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

In [25]:
y_prediction = model.predict(X_test)

In [26]:
y_prediction.reshape(-1,1)

array([[1.],
       [0.],
       [1.],
       ...,
       [0.],
       [1.],
       [0.]], dtype=float32)

In [40]:
scores = model.evaluate(X_test, y_prediction)

282/282 [==============================] - 0s 1ms/step - loss: 0.0036 - accuracy: 0.9997


In [ ]:
#print("Recall score:"+ str(recall_score(y_test, y_prediction, average# != 'binary')))

In [ ]:
#print(classification_report(y_test, y_prediction,
                            target_names=["default", "non_default"]))

In [37]:
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="red" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_prediction)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Non_defualt', 'Default'],
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Non_defualt', 'Default'], normalize=True,
                      title='Normalized confusion matrix')

plt.show()